In [1]:
from DTG.classes.LLM import *
from DTG.prompts import prompt_funcs
import json, time

In [2]:
INPUT_INFO = 'NPC personality : lazy and shy . NPC mood : confused and amused. Convesation features : dancing, great move, tripping'

In [3]:
def get_dialogue_json(input_info):
    t_0 = time.time()
    starting_time = time.time()
    performance_dict = {
        'tags' : 0,
        'abstract' : 0,
        'endings' : 0,
        'tree' : 0,
        'formatted_json' : 0
    }
    llm = LLM()
    print(f'Generating tags.')
    tags_by_llm = llm.send_message_for_code(prompt_funcs.tag_prompt(input_info), model='gpt-4o-mini')
    print(f'Tags generated in {time.time() - starting_time} seconds.')
    performance_dict['tags'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating abstract.')
    abstract = llm.send_message_for_code(prompt_funcs.abstract_prompt(tags_by_llm, input_info))
    print(f'Abstract generated in {time.time() - starting_time} seconds.')
    performance_dict['abstract'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating possible endings.')
    endings_by_llm = llm.send_message_for_code(prompt_funcs.possible_ending_prompt(abstract), model='o1-mini')
    print(f'Possible endings generated in {time.time() - starting_time} seconds.')
    performance_dict['endings'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating dialogue tree.')
    tree_by_llm = llm.send_message_for_code(prompt_funcs.dialogue_prompt(abstract, endings_by_llm, tags_by_llm), model='o1-preview')
    print(f'Dialogue tree generated in {time.time() - starting_time} seconds.')
    performance_dict['tree'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Formatting JSON.')
    formatted_tree = llm.send_message(prompt_funcs.format_prompt(tree_by_llm), model='gpt-4o-mini')
    print(f'JSON formatted in {time.time() - starting_time} seconds.')
    performance_dict['formatted_json'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Total time taken : {time.time() - t_0} seconds.')
    print(f'Performance dict : {performance_dict}')
    formatted_tree_json = json.loads(formatted_tree)
    #formatted_endings_json = json.loads(endings_by_llm)
    return formatted_tree_json, endings_by_llm


In [4]:
#tree, endings = get_dialogue_json(INPUT_INFO)

In [18]:
from DTG.classes.OutputClasses import Tree, PossibleEndings

In [19]:
def get_dialogue_json(input_info):
    t_0 = time.time()
    starting_time = time.time()
    performance_dict = {
        'tags' : 0,
        'abstract' : 0,
        'endings' : 0,
        'tree' : 0
    }
    llm = LLM()
    print(f'Generating tags.')
    tags_by_llm = llm.send_message_for_code(prompt_funcs.tag_prompt(input_info), model='gpt-4o-mini')
    print(f'Tags generated in {time.time() - starting_time} seconds.')
    performance_dict['tags'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating abstract.')
    abstract = llm.send_message_for_code(prompt_funcs.abstract_prompt(tags_by_llm, input_info))
    print(f'Abstract generated in {time.time() - starting_time} seconds.')
    performance_dict['abstract'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating possible endings.')
    endings_by_llm = llm.send_message_for_format(prompt_funcs.possible_ending_prompt(abstract), PossibleEndings ,model='gpt-4o')
    print(f'Possible endings generated in {time.time() - starting_time} seconds.')
    performance_dict['endings'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating dialogue tree.')
    tree_by_llm = llm.send_message_for_format(prompt_funcs.dialogue_prompt(abstract, endings_by_llm, tags_by_llm), format=Tree,model='gpt-4o')
    print(f'Dialogue tree generated in {time.time() - starting_time} seconds.')
    performance_dict['tree'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Total time taken : {time.time() - t_0} seconds.')
    print(f'Performance dict : {performance_dict}')
    #formatted_endings_json = json.loads(endings_by_llm)
    return tree_by_llm, endings_by_llm


In [20]:
tree, endings = get_dialogue_json(INPUT_INFO)

Generating tags.
Tags generated in 2.6880886554718018 seconds.
Generating abstract.
Abstract generated in 3.8903868198394775 seconds.
Generating possible endings.
Possible endings generated in 4.215151071548462 seconds.
Generating dialogue tree.
Dialogue tree generated in 24.871508836746216 seconds.
Total time taken : 35.665934801101685 seconds.
Performance dict : {'tags': 2.688295841217041, 'abstract': 3.890544891357422, 'endings': 4.2153661251068115, 'tree': 24.8717200756073}


In [17]:
with open('output.json', 'w') as file:
    json.dump(json.loads(tree), file, indent=4)